In [1]:
import socket
import numpy as np
import pandas as pd
import platform
import logging

import common



In [46]:
def get_CPU_data(cpu_name):
    if(cpu_name == "AMD Ryzen 7 3700X 8-Core Processor"):
        return {
            "cpu_price": 350.0,
            "cpu_power": 65.0,
            "n_threads": 8
        }
    
    if(cpu_name == "Intel(R) Xeon(R) Gold 6248 CPU @ 2.50GHz"): 
        return {
            "cpu_price": 3290.0,
            "cpu_power": 150.0,
            "n_threads": 40
        }

    else: 
        logging.warning(f"Unknown CPU {cpu_name}")
        return {}


def get_GPU_data(gpu_name):
    if(gpu_name == "NVIDIA GeForce GTX 970"):
        return {
            "gpu_price": 260.0,
            "gpu_power": 500.0,
            "gpu_grid_size": 256,
            "gpu_block_size": 256
        }
    if(gpu_name == "Tesla T4"):
        return {
            "gpu_price": 2500.0,
            "gpu_power": 70.0,
            "gpu_grid_size": 256,
            "gpu_block_size": 256
        }
    

def get_system_information():
    cpu_name = cpuinfo.get_cpu_info()["brand_raw"]
    gpu_name = os.popen("nvidia-smi --query-gpu=name --format=csv").read().split("\n")[1]

    data = {
        "host_name" : socket.gethostname(),
        "cpu_name": cpu_name,
        "gpu_name": gpu_name
    }

    data.update(get_CPU_data(cpu_name))
    data.update(get_GPU_data(gpu_name))
    return data




get_system_information()

{'host_name': 'Hiti-Work',
 'cpu_name': 'AMD Ryzen 7 3700X 8-Core Processor',
 'gpu_name': 'NVIDIA GeForce GTX 970',
 'cpu_price': 350.0,
 'cpu_power': 65.0,
 'gpu_price': 260.0,
 'gpu_power': 500.0}